In [103]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
from scipy.stats.stats import ttest_ind
from scipy.stats.mstats import winsorize
from sqlalchemy import create_engine
from scipy.stats import boxcox
import warnings

In [ ]:
# Database credentials
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'useducation'

# use credentials to start a connection
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(postgres_user, 
    postgres_pw, postgres_host, postgres_port, postgres_db))

df_useducation = pd.read_sql_query('select * from useducation',con=engine)
engine.dispose()

THIS WAS NEEDED AS BACKUP WHEN INTERNET WENT DOWN.
df_useducation = pd.read_csv('states_all.csv')

In [105]:
df_useducation.head()

,PRIMARY_KEY,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,...,GRADES_4_G,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE
0,1992_ALABAMA,ALABAMA,1992,NaN,2678885.0,304177.0,1659028.0,715680.0,2653798.0,1481703.0,...,57948.0,58025.0,41167.0,NaN,NaN,731634.0,208.327876,252.187522,207.963517,NaN
1,1992_ALASKA,ALASKA,1992,NaN,1049591.0,106780.0,720711.0,222100.0,972488.0,498362.0,...,9748.0,8789.0,6714.0,NaN,NaN,122487.0,NaN,NaN,NaN,258.859712
2,1992_ARIZONA,ARIZONA,1992,NaN,3258079.0,297888.0,1369815.0,1590376.0,3401580.0,1435908.0,...,55433.0,49081.0,37410.0,NaN,NaN,673477.0,215.253932,265.366278,206.212716,262.169895
3,1992_ARKANSAS,ARKANSAS,1992,NaN,1711959.0,178571.0,958785.0,574603.0,1743022.0,964323.0,...,34632.0,36011.0,27651.0,NaN,NaN,441490.0,210.206028,256.312090,208.634458,264.619665
4,1992_CALIFORNIA,CALIFORNIA,1992,NaN,26260025.0,2072470.0,16546514.0,7641041.0,27138832.0,14358922.0,...,418418.0,363296.0,270675.0,NaN,NaN,5254844.0,208.398961,260.892247,196.764414,NaN


Say, we want to understand the relationship between the expenditures of the governments and the students' overall success in the math and reading.

1. Create a new score variable from the weighted averages of all score variables in the datasets. **Notice that the number of students in the 4th grade isn't the same as the number of students in the 8th grade. So, you should appropriately weigh the scores!**.
2. What are the correlations between this newly created score variable and the expenditure types? Which 1 of the expenditure types is more correlated than the others?
3. Now, apply PCA to the 4 expenditure types. How much of the total variance is explained by the 1st component?
4. What is the correlation between the overall score variable and the 1st principal component? 
5. If you were to choose the best variables for your model, would you prefer using the 1st principal component instead of the expenditure variables? Why?

### 1. Create a new score variable from the weighted averages of all score variables in the datasets. **Notice that the number of students in the 4th grade isn't the same as the number of students in the 8th grade. So, you should appropriately weigh the scores!**.

In [106]:
# Checking to see if there's missing data

# Lots of missing data!! Gonna interpolate data because it's time series.
df_useducation.isna().sum()

PRIMARY_KEY                        0
STATE                              0
YEAR                               0
ENROLL                           694
TOTAL_REVENUE                    643
FEDERAL_REVENUE                  643
STATE_REVENUE                    643
LOCAL_REVENUE                    643
TOTAL_EXPENDITURE                643
INSTRUCTION_EXPENDITURE          643
SUPPORT_SERVICES_EXPENDITURE     643
OTHER_EXPENDITURE                694
CAPITAL_OUTLAY_EXPENDITURE       643
GRADES_PK_G                      376
GRADES_KG_G                      286
GRADES_4_G                       286
GRADES_8_G                       286
GRADES_12_G                      286
GRADES_1_8_G                     898
GRADES_9_12_G                    847
GRADES_ALL_G                     286
AVG_MATH_4_SCORE                1383
AVG_MATH_8_SCORE                1387
AVG_READING_4_SCORE             1386
AVG_READING_8_SCORE             1421
dtype: int64

In [107]:
fill_list = ["ENROLL", "TOTAL_REVENUE", "FEDERAL_REVENUE",
             "STATE_REVENUE", "LOCAL_REVENUE", "TOTAL_EXPENDITURE",
             "INSTRUCTION_EXPENDITURE", "SUPPORT_SERVICES_EXPENDITURE",
             "OTHER_EXPENDITURE", "CAPITAL_OUTLAY_EXPENDITURE", "GRADES_PK_G",
             "GRADES_KG_G", "GRADES_4_G", "GRADES_8_G", "GRADES_12_G", "GRADES_1_8_G",
             "GRADES_9_12_G", "GRADES_ALL_G"]

states = df_useducation["STATE"].unique()

for state in states:
   df_useducation.loc[df_useducation["STATE"] == state, fill_list] = df_useducation.loc[df_useducation["STATE"] == state, fill_list].interpolate()

# we drop the null values after interpolation
df_useducation.dropna(inplace=True)

In [108]:
# Confirm no more nulls
df_useducation.isna().sum()

PRIMARY_KEY                     0
STATE                           0
YEAR                            0
ENROLL                          0
TOTAL_REVENUE                   0
FEDERAL_REVENUE                 0
STATE_REVENUE                   0
LOCAL_REVENUE                   0
TOTAL_EXPENDITURE               0
INSTRUCTION_EXPENDITURE         0
SUPPORT_SERVICES_EXPENDITURE    0
OTHER_EXPENDITURE               0
CAPITAL_OUTLAY_EXPENDITURE      0
GRADES_PK_G                     0
GRADES_KG_G                     0
GRADES_4_G                      0
GRADES_8_G                      0
GRADES_12_G                     0
GRADES_1_8_G                    0
GRADES_9_12_G                   0
GRADES_ALL_G                    0
AVG_MATH_4_SCORE                0
AVG_MATH_8_SCORE                0
AVG_READING_4_SCORE             0
AVG_READING_8_SCORE             0
dtype: int64

In [109]:
# Use weighted average of all scores. New variable is wt_avg_scores
df_useducation["wt_avg_scores"] = (df_useducation["GRADES_4_G"]*((df_useducation["AVG_MATH_4_SCORE"] + df_useducation["AVG_READING_4_SCORE"])*0.5) + df_useducation["GRADES_8_G"]
                                 * ((df_useducation["AVG_MATH_8_SCORE"] + df_useducation["AVG_READING_8_SCORE"])*0.5))/(df_useducation["GRADES_4_G"] + df_useducation["GRADES_8_G"])

df_useducation.head()

,PRIMARY_KEY,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,...,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE,wt_avg_scores
408,2000_ALABAMA,ALABAMA,2000,730184.0,4885917.0,425978.0,3003814.0,1456125.0,5003267.0,2551713.0,...,56951.0,42512.0,472686.0,201358.0,739992.0,217.941053,262.158206,206.880030,258.777258,235.863027
410,2000_ARIZONA,ARIZONA,2000,814807.0,5479168.0,534619.0,2481089.0,2463460.0,5397194.0,2488932.0,...,65526.0,49501.0,567626.0,236933.0,877696.0,218.766619,270.715406,205.347275,270.519439,239.899103
411,2000_ARKANSAS,ARKANSAS,2000,450751.0,2844855.0,254735.0,2126099.0,464021.0,2683400.0,1531232.0,...,34873.0,28918.0,281091.0,131511.0,449959.0,217.062870,261.363713,212.881797,268.004817,239.528677
412,2000_CALIFORNIA,CALIFORNIA,2000,5941946.0,46628207.0,4024940.0,26877920.0,15725347.0,45829753.0,23514388.0,...,441877.0,357789.0,3804640.0,1707952.0,6140814.0,213.565125,262.171027,205.916588,264.494066,235.179063
414,2000_CONNECTICUT,CONNECTICUT,2000,530363.0,5884078.0,234321.0,2298931.0,3350826.0,5605280.0,3189123.0,...,42597.0,32591.0,354391.0,155734.0,562179.0,234.243375,281.902486,229.381143,266.468618,252.492779


### 2. What are the correlations between this newly created score variable and the expenditure types? Which 1 of the expenditure types is more correlated than the others?

In [110]:
df_useducation[['wt_avg_scores', 'TOTAL_EXPENDITURE', 'INSTRUCTION_EXPENDITURE',
              'SUPPORT_SERVICES_EXPENDITURE', 'OTHER_EXPENDITURE', 'CAPITAL_OUTLAY_EXPENDITURE']].corr()

,wt_avg_scores,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE
wt_avg_scores,1.000000,-0.036699,-0.023164,-0.034972,-0.097609,-0.094862
TOTAL_EXPENDITURE,-0.036699,1.000000,0.997494,0.996012,0.975813,0.936234
INSTRUCTION_EXPENDITURE,-0.023164,0.997494,1.000000,0.995281,0.970818,0.916236
SUPPORT_SERVICES_EXPENDITURE,-0.034972,0.996012,0.995281,1.000000,0.966962,0.913193
OTHER_EXPENDITURE,-0.097609,0.975813,0.970818,0.966962,1.000000,0.927021
CAPITAL_OUTLAY_EXPENDITURE,-0.094862,0.936234,0.916236,0.913193,0.927021,1.000000


In [111]:
df_useducation[['wt_avg_scores', 'TOTAL_EXPENDITURE', 'INSTRUCTION_EXPENDITURE',
              'SUPPORT_SERVICES_EXPENDITURE', 'OTHER_EXPENDITURE', 'CAPITAL_OUTLAY_EXPENDITURE']].corr()

,wt_avg_scores,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE
wt_avg_scores,1.000000,-0.036699,-0.023164,-0.034972,-0.097609,-0.094862
TOTAL_EXPENDITURE,-0.036699,1.000000,0.997494,0.996012,0.975813,0.936234
INSTRUCTION_EXPENDITURE,-0.023164,0.997494,1.000000,0.995281,0.970818,0.916236
SUPPORT_SERVICES_EXPENDITURE,-0.034972,0.996012,0.995281,1.000000,0.966962,0.913193
OTHER_EXPENDITURE,-0.097609,0.975813,0.970818,0.966962,1.000000,0.927021
CAPITAL_OUTLAY_EXPENDITURE,-0.094862,0.936234,0.916236,0.913193,0.927021,1.000000


In this case, the most correlated expenditure with weighted average of scores is instruction expenditure.

### 3. Now, apply PCA to the 4 expenditure types. How much of the total variance is explained by the 1st component?

In [112]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

X = df_useducation[["INSTRUCTION_EXPENDITURE", "SUPPORT_SERVICES_EXPENDITURE",
                  "OTHER_EXPENDITURE", "CAPITAL_OUTLAY_EXPENDITURE"]]

X = StandardScaler().fit_transform(X)

sklearn_pca = PCA(n_components=1)
df_useducation["pca_1"] = sklearn_pca.fit_transform(X)

print('About ' +str(sklearn_pca.explained_variance_ratio_) + ' of the total variance is explained by the 1st component.')

About [0.96136563] of the total variance is explained by the 1st component.


### 4. What is the correlation between the overall score variable and the 1st principal component? 

In [113]:
df_useducation[['wt_avg_scores', 'pca_1', "TOTAL_EXPENDITURE", "INSTRUCTION_EXPENDITURE",
              "SUPPORT_SERVICES_EXPENDITURE", "OTHER_EXPENDITURE", "CAPITAL_OUTLAY_EXPENDITURE"]].corr()

,wt_avg_scores,pca_1,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE
wt_avg_scores,1.000000,-0.063586,-0.036699,-0.023164,-0.034972,-0.097609,-0.094862
pca_1,-0.063586,1.000000,0.996057,0.990256,0.988508,0.985626,0.957209
TOTAL_EXPENDITURE,-0.036699,0.996057,1.000000,0.997494,0.996012,0.975813,0.936234
INSTRUCTION_EXPENDITURE,-0.023164,0.990256,0.997494,1.000000,0.995281,0.970818,0.916236
SUPPORT_SERVICES_EXPENDITURE,-0.034972,0.988508,0.996012,0.995281,1.000000,0.966962,0.913193
OTHER_EXPENDITURE,-0.097609,0.985626,0.975813,0.970818,0.966962,1.000000,0.927021
CAPITAL_OUTLAY_EXPENDITURE,-0.094862,0.957209,0.936234,0.916236,0.913193,0.927021,1.000000


### 5. If you were to choose the best variables for your model, would you prefer using the 1st principal component instead of the expenditure variables? Why?

Both have the benefits and setbacks. In this case, normalizing the data to get a weighted average by manual means can add a source of error in arithmatic processes. Quick and easy method would be to use the PCA method and they both seem very similar in results.